# Importation Des Librairies

In [1]:
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import collections
from sklearn.linear_model import LinearRegression
import numpy as np
import collections
from skimage.transform import resize

# fonction qui divise an array 

In [3]:

def blockshaped(arr, nrows, ncols):
    return (arr.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))

# Prediction

In [15]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
def ReLU(Z):
    return np.maximum(Z, 0)    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2
def get_predictions(A2):
    return np.argmax(A2, 0)
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    max=np.max(A2)
    return predictions,max
def test_prediction(image, W1, b1, W2, b2):
    prediction,max = make_predictions(image, W1, b1, W2, b2)
    prediction=prediction[0]
    return prediction,max

# features extraction pour image inserer par user

In [16]:
def extract_image(img):
    features = []
    out=[]
    b_pentes = []
    first_feature = []
    deuxieme_feature = []
    troisieme_feature = []
    df = pd.DataFrame(img)
    df=df.loc[(df != 0).any(axis=1), (df != 0).any(axis=0)]
    data=df.to_numpy()
    img = resize(data, (28,28),anti_aliasing=True)
### extraction de la pente b de toutes les sousmatrices############
    for k in range(0,16):
        flat_numpy = numpy.array(blockshaped(img,7,7)[k].flatten())
        nb_blackcell_parcell = (49-collections.Counter(flat_numpy)[0])/49
        first_feature.append(nb_blackcell_parcell)
        x_data = []
        y_data = []
        for i in range(0,7):
            for j in range(0,7):
                if(blockshaped(img,7,7)[k][i,j]>0):
                    x_data.append(i)
                    y_data.append(j)
                    #z = np.polyfit(x_data, y_data, 1)
                    a=numpy.array(x_data).reshape(-1,1)
                    a=a.reshape(-1,1)
                    model = LinearRegression()
                    model.fit(a,y_data)
                    pred = model.predict(a)
        b_pentes.append(model.coef_[0])
        deuxieme_feature =[((2*i)/(1+i*i)) for i in b_pentes]
        troisieme_feature = [(1-i*i)/(1+i*i) for i in b_pentes]
    out.append(first_feature+deuxieme_feature+troisieme_feature)#+[downloaded_test[5999][1]])
    matrice= np.array(out[0]).reshape(1,-1)
    #numpy.savetxt(r'C:\Users\taoul\OneDrive\Bureau\image9.csv', matrice, delimiter=",")
    return matrice

# Chargement de W1, b1,W2,b2

In [17]:
from numpy import genfromtxt
W1 = genfromtxt(r'W1.csv', delimiter=',')
b1 = genfromtxt(r'b1.csv', delimiter=',')
W2 = genfromtxt(r'W2.csv', delimiter=',')
b2 = genfromtxt(r'b2.csv', delimiter=',')

In [20]:
b2 =b2.reshape(-1,1)
b1 = b1.reshape(-1,1)

# Interface graphique

In [21]:
from tkinter import *
import tkinter.font as tkFont
import win32gui
from PIL import ImageGrab, Image
import numpy as np


'''
Let us deploy the model in the python application using tkinter
'''
class application(Frame):
    def __init__(self,master):
        super().__init__(master)
        self.fontStyle = tkFont.Font(family="Lucida Grande", size=20)
        self.master=master
        self.pack()
        self.createWidget()

    def createWidget(self):
        self.canvas = Canvas(self,width=224,height=224,bg='black')
        self.canvas.pack(expand=YES, fill=BOTH)
        self.canvas.bind('<B1-Motion>',self.activate_paint)

    def activate_paint(self, event):
        global lastx, lasty
        self.canvas.bind('<B1-Motion>', self.paint)
        lastx, lasty = event.x, event.y

    def paint(self,event):
        global lastx, lasty
        x, y = event.x, event.y
        self.canvas.create_line((lastx, lasty, x, y), width=12, fill='white')

        lastx, lasty = x, y

    def clearCanvas(self):
        self.canvas.delete("all")
        answer.configure(text='Answer Goes Here', font=self.fontStyle)
        self.canvas.bind('<B1-Motion>',self.activate_paint)

    def predicted_result(self, data):
        ans = data.argsort()[-8:][::-1]  #sorting in descending
        return ans


    def predictDigit(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        im = ImageGrab.grab(rect) # get image of the current location
        im.save('file.png')

        img = Image.open('file.png').convert('L')
        img = img.resize((28,28), Image.ANTIALIAS)
        img.save('resized.png')

        #after resizing the image data
        #convert to np array

        data = np.array(img)
        for i in range(0,28):
            for j in range(0,28):
                if data[i][j]<90:
                    data[i][j]=0
                
        image = extract_image(data).T
        predictionk,proba= test_prediction(image, W1, b1, W2, b2)

        #import the model
        
        answer.configure(text='Predicted Digit: '+str(predictionk)+'\nWith probability :'+str(round(proba,2)), font=self.fontStyle)

        
if __name__ == '__main__':
    root = Tk()
    root.geometry('300x400')
    
    
    app=application(root)
    # let us create button to clear the drawn items and predict the value

    clear = Button(root, text='Clear', command=app.clearCanvas)
    clear.pack()

    predict = Button(root, text='Predict', command=app.predictDigit)
    predict.pack()

    answer = Label(root, text="Answer Goes Here", font=app.fontStyle)
    answer.pack()

    root.title('Draw a Digit')
    root.mainloop()